In [ ]:
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np
from IPython.display import clear_output
import time
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import linkage, cophenet


DATA_PATH = r'C:\Users\paul_\OneDrive\Pro\Advestis\K determination\analysis\datasets'

In [28]:
df = pd.read_csv(DATA_PATH + r'\swimmer\swimmer_2.csv')
m0 = df.values[:,2:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\sausage\Sausage Raw NIR.csv')
# m0 = df.values[:,8:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\esg\esg_posneg.csv')
# m0 = df.values[:,1:].astype(np.float_)

# df = pd.read_csv(DATA_PATH + r'\brunet\ALL-AML Brunet.csv')
# m0 = df.values[:,2:].astype(np.float_)

(n,p) = np.shape(m0)

# for Brunet only:
# m0 = np.log2(df.values[:,2:].astype(np.float_))
# m0-=np.repeat(np.min(m0, axis=0)[:,np.newaxis].T, n, axis=0)

min_comp = 10
max_comp = 20
n_runs = 50
iter_max = 10

In [29]:
time_start = time.time()
test_w = np.zeros(max_comp)
test_h = np.zeros(max_comp)
c_w = np.zeros(n)
c_h = np.zeros(p)
iln1 = np.triu_indices(n,1)
ilp1 = np.triu_indices(p,1)

for n_comp in range(min_comp,max_comp+1):
    my_nmfmodel = NMF(n_components=n_comp, init='nndsvda', solver='cd', beta_loss='frobenius', max_iter=iter_max, random_state=0)
    w4 = my_nmfmodel.fit_transform(m0)
    h4 = my_nmfmodel.components_.T
    error = np.linalg.norm(m0 - w4 @ h4.T)

    co_w = np.zeros(int(n*(n-1)/2))
    co_h = np.zeros(int(p*(p-1)/2))
    my_nmfmodel_random = NMF(n_components=n_comp, init='custom', solver='cd', beta_loss='frobenius', max_iter=iter_max, random_state=0)
    for i_run in range(0,n_runs):
        print('n_comp = ' + str(n_comp) + '; i_run = ' + str(i_run))
        w4_init = np.random.rand(n,n_comp); h4_init = np.random.rand(p,n_comp).T
        w4 = my_nmfmodel_random.fit_transform(m0, W=w4_init, H=h4_init)
        h4 = my_nmfmodel_random.components_.T
        c_w = np.argmax(normalize(w4, axis=0), axis=1)
        c_h = np.argmax(normalize(h4, axis=0), axis=1)
        # co_w += np.array([c_w[i] == c_w[j] for i in range(0,n-1) for j in range(i+1,n)])
        # co_h += np.array([c_h[i] == c_h[j] for i in range(0,p-1) for j in range(i+1,p)])
        co_w += np.equal.outer(c_w, c_w)[iln1]
        co_h += np.equal.outer(c_h, c_h)[ilp1]

    co_w = 1 - co_w/n_runs
    co_h = 1 - co_h/n_runs
    cpc_w, cp_w = cophenet(linkage(co_w, method='ward'), co_w)
    cpc_h, cp_h = cophenet(linkage(co_h, method='ward'), co_h)
    clear_output(wait=True)
    test_w[n_comp-1] = cpc_w / error
    test_h[n_comp-1] = cpc_h / error

time_elapsed = (time.time() - time_start)
print(time_elapsed)

49.76531171798706


In [30]:
# Cusum calculation
cusum = np.zeros(max_comp)
test = np.sqrt(test_w * test_h)
cusum[min_comp-2] = (test[min_comp-2]-test[min_comp-1] > 0)

for n_comp in range(min_comp,max_comp):
    if test[n_comp-1]-test[n_comp] > 0:
        deltax = 1
    else:
        deltax = -1
    
    cusum[n_comp-1] = max(cusum[n_comp-2]+deltax, 0)

cusum[max_comp-1] = cusum[max_comp-2]

# Rank estimation
n_comp_est = 999
for n_comp in range(1,max_comp-2):
    if cusum[n_comp-1]*cusum[n_comp]*cusum[n_comp+1]> 0:
        n_comp_est = n_comp
        break

print('Estimated rank = ', n_comp_est)

Estimated rank =  17


In [ ]:
test_w

In [ ]:
# np.savetxt(DATA_PATH + r'\brunet\test_scipy_cd_brunet.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
# np.savetxt(DATA_PATH + r'\sausage\test_scipy_cd_sausage.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
np.savetxt(DATA_PATH + r'\swimmer\test_scipy_cd_swimmer.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')
# np.savetxt(DATA_PATH + r'\esg\test_scipy_cd_esg.csv',  np.concatenate((test_w, test_h), axis=0), delimiter=',')


In [ ]:
np.savetxt(DATA_PATH + r'\foo.csv', 1-co_h, delimiter=',')